In [1]:
from data_load import load_query_games, load_query_games, load_system_prompt
from utils import get_model_response
import pandas as pd

In [2]:
games = load_query_games()
sg = games[0]

In [3]:
t = sg.tasks[0]
print(list(t.keys()))
print(t['query_list'])
print(t['query'])
print(t['answer'])


['all_moves', 'query', 'query_code', 'answer', 'lang_phrase_list', 'code_phrase_list', 'query_list']
[Text: Player 1 moves their piece from 1st row, 4th column to 2nd row, 4th column.
Code: (Move D4-D3), Text: Player 2 moves their piece from 1st row, 3rd column to 1st row, 4th column.
Code: (Move C4-D4), Text: Player 1 moves their piece from 1st row, 2nd column to 2nd row, 2nd column.
Code: (Move B4-B3), Text: Player 2 moves their piece from 4th row, 2nd column to 3rd row, 2nd column.
Code: (Move B1-B2), Text: Player 1 moves their piece from 4th row, 3rd column to 3rd row, 3rd column.
Code: (Move C1-C2), Text: Player 2 moves their piece from 4th row, 4th column to 3rd row, 4th column.
Code: (Move D1-D2), Text: Player 1 moves their piece from 3rd row, 3rd column to 2nd row, 3rd column.
Code: (Move C2-C3)]
who wins?
Player 1 wins because three of their pieces form a line on the 2nd row.


In [4]:
def get_all_task_messages(game, task_id):
    messages = []
    task = game.tasks[task_id]
    query_list = task['query_list']

    messages = []

    messages.append({"role": "system", "content": load_system_prompt('new_system_prompt.txt')})
    messages.append({"role": "user", "content": f"Name: {game.name}\nDescription: {game.description}"})
    messages.append({"role": "assistant", "content": game.code})

    
    messages.append({"role": "system", "content": load_system_prompt('query_prompt.txt')})

    for query in query_list:
        messages.append({"role": "user", "content": query.text})
        messages.append({"role": "assistant", "content": query.code})

    messages.append({"role": "system", "content": load_system_prompt('answer_prompt.txt')})
    messages.append({"role": "user", "content": task['query']})

    return messages



In [9]:
for game in games:
    game_name = game.tag
    print(f"{game_name} is being processed..")
    save_path = 'data/predicted_answer_model/' + game_name + '.csv'
    model_response = []
    answers = []
    task_ids = []
    for task_id in range(len(game.tasks)):
        messages = get_all_task_messages(game, task_id)
        model_response.append(get_model_response(messages))
        answers.append(game.tasks[task_id]['answer'])
        task_ids.append(task_id)
    df = pd.DataFrame({'model_response': model_response, 'answer': answers})
    df.to_csv(save_path, index=False)

asimplegame is being processed..
tictactoe3d is being processed..
achi is being processed..
doguti is being processed..
